# Init Process

## Init01 Import Package

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Init02. Read DataSet

In [9]:

train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

print("train 데이터 크기 : %s 행 %s 열" % train_data.shape)
train_data.head(3)


train 데이터 크기 : 114 행 11 열


,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854,4400


## Init03. Check Missing Data

In [17]:
# pd.isnull(train_data)
print("Train - 결측값 개수")
display(train_data.isnull().sum())
print("--------------------------")
print("Test - 결측값 개수")
display(test_data.isnull().sum())


Train - 결측값 개수


id                     0
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    3
Delta 15 N (o/oo)      3
Delta 13 C (o/oo)      3
Body Mass (g)          0
dtype: int64

--------------------------
Test - 결측값 개수


id                     0
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    6
Delta 15 N (o/oo)      9
Delta 13 C (o/oo)      8
dtype: int64

## Init04 Visualize Data
결측값과 연관이 있을법한 data를 axis로 설정하고   
결측값에 해당하는 열을 color로 표시하여 시각화하기  
결측값 예측에 대한 ML 만들어서 결측값 체워넣기

## Init05 Fill Missing Data

# Main Process

## Process01 Visulize Data

# Process02 Interpretation

## Process03 Train ML

# End Process

## End01 - Accuracy Verification

## End02 - Write to CSV